In [0]:
!pip install -U -q PyDrive
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [0]:
DATA_FOLDER_ID = '1UnlCdTU78CAuIRs7Lrp8YAIh8hS31z4A'
ROOT_PATH = 'Datasets'




In [0]:


# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)




In [0]:
local_root_path = os.path.expanduser(ROOT_PATH)
try:
  os.makedirs(local_root_path)
except: pass

In [0]:
def ListFolder(google_drive_id, destination):
  file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % google_drive_id}).GetList()
  counter = 0
  for f in file_list:
    # If it is a directory then, create the dicrectory and upload the file inside it
    if f['mimeType']=='application/vnd.google-apps.folder': 
      folder_path = os.path.join(destination, f['title'])
      os.makedirs(folder_path)
      print('creating directory {}'.format(folder_path))
      ListFolder(f['id'], folder_path)
    else:
      fname = os.path.join(destination, f['title'])
      f_ = drive.CreateFile({'id': f['id']})
      f_.GetContentFile(fname)
      counter += 1
  print('{} files were uploaded in {}'.format(counter, destination))

In [20]:
ListFolder(DATA_FOLDER_ID, local_root_path)

FileExistsError: ignored

In [0]:
DATADIR = 'Datasets/HandPhotos'
CATEGORIES = ['C', 'O']

In [0]:
IMG_SIZE = 100
def create_training_data():
    training_data = []
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=C 1=O

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
              
    return training_data

In [40]:
training_data = create_training_data()


100%|██████████| 224/224 [00:00<00:00, 980.95it/s]


In [41]:
import random

print(len(training_data))
random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

466
0
0
1
0
0
1
1
0
0
0


In [0]:
X = []
Y = []

for features,label in training_data:
    X.append(features)
    Y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X = X/255.0

In [0]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [45]:
model.fit(X, Y, batch_size=32, epochs=10, validation_split=0.3)

Train on 326 samples, validate on 140 samples
Epoch 1/10
326/326 [==============================] - 3s 9ms/step - loss: 1.4529 - acc: 0.6012 - val_loss: 0.5285 - val_acc: 0.8357
Epoch 2/10
326/326 [==============================] - 2s 5ms/step - loss: 0.3276 - acc: 0.8804 - val_loss: 0.0818 - val_acc: 0.9857
Epoch 3/10
326/326 [==============================] - 2s 5ms/step - loss: 0.0656 - acc: 0.9816 - val_loss: 0.0243 - val_acc: 0.9929
Epoch 4/10
326/326 [==============================] - 2s 5ms/step - loss: 0.0424 - acc: 0.9847 - val_loss: 0.0265 - val_acc: 0.9929
Epoch 5/10
326/326 [==============================] - 2s 5ms/step - loss: 0.0180 - acc: 0.9939 - val_loss: 0.0070 - val_acc: 1.0000
Epoch 6/10
326/326 [==============================] - 2s 5ms/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0027 - val_acc: 1.0000
Epoch 7/10
326/326 [==============================] - 2s 5ms/step - loss: 5.5900e-04 - acc: 1.0000 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 8/10
326/326 [======

In [0]:
model.save('Hand-CNN-v2-CO.model')

In [0]:
upload = drive.CreateFile({'title': 'Hand-CNN-v2-CO.model'})
upload.SetContentFile('Hand-CNN-v2-CO.model')
upload.Upload()

In [51]:
!ls


adc.json  Hand-CNN-v2-colored.model  sample_data
Datasets  Hand-CNN-v2-CO.model
